In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from feature_learning import *
from load_and_preprocessing import load_data_and_filter_members

In [ ]:
leg='50'
from_date = None
to_date = None
link_cutoff = 0.5
data_transf, adjacency, node_index, affairs_features, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=True, delete_links_below=link_cutoff )

Put train data import here after

In [ ]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

X_train = X_train[y_train != 2]
y_train = y_train[y_train !=2]

X_train = GFT(X_train.T)

X_train = X_train.T

print("Train data (n_samples, n_dimensions): {0}".format(X_train.shape))
classes, counts = np.unique(y_train, return_counts=True)
print("Classes {0}, samples per class {1}, {2}"
      .format(len(classes), counts[0], counts[1]))

In [ ]:
graph = nx.from_numpy_array(adjacency)
laplacian = nx.normalized_laplacian_matrix(graph)
vals, vecs = np.linalg.eigh(laplacian.toarray())
e = vals
U = vecs

X_f = (GFT(U, X_train)).T
X_f_heat = graph_filter()

**Do PCA**

In [ ]:
#X = do_pca(X_train, n_comps=10, plot_expl_var=True)
X = X_train
X.shape

**Do logistic regression**

In [ ]:
log_reg_accuracy = fit_log_regression(X,y_train,20,k_fold=5)

**Do SVM**

In [ ]:
c_hyperparam = np.logspace(-6,-1,11)
use_kernel = 'linear'
k_fold = 5

mean_accuracy, variance = fit_svm(X, y_train, use_kernel, c_hyperparam, k_fold=k_fold)

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(mean_accuracy, 'b.-')
ax1.set_ylabel('Accuracy', color='b')
ax2 = ax1.twinx()
ax2.plot(variance, 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

ConvNN:

In [ ]:
leg='50'
from_date = None
to_date = None
link_cutoff = 0.7
data_transf, adjacency, node_index, affairs_features, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=True, delete_links_below=link_cutoff )

In [ ]:
hyperparam_values = np.logspace(-7,-2,15)
hyperparam_name = 'regularization'
k_fold = 5
nn_accuracy, nn_loss = cross_validate_convNN(X_train, y_train, adjacency, hyperparam_name, hyperparam_values, k_fold)

In [ ]:
nn_accuracy

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(hyperparam_values, np.array(nn_accuracy)[:,1], 'b.-')
ax1.set_ylabel('Accuracy', color='b')
ax1.set_xlabel(hyperparam_name)
ax2 = ax1.twinx()
ax2.plot(hyperparam_values, np.array(nn_loss)[:,1], 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

In [ ]:
hyperparam_values